In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.ops import unary_union

from skimage.metrics import (
    mean_squared_error, 
    structural_similarity as ssim, 
    peak_signal_noise_ratio
)

matplotlib.use('Agg')  # for saving figures

MIN_AREA = 50
COLOR_DIST_THRESHOLD = 30
WIN_SIZE_FOR_SSIM = 3

###############################################################################
# Parsing the Real Image (Exact Colors)
###############################################################################
def parse_color_coded_image(image_rgb, min_area=MIN_AREA):
    """
    Exactly parse each unique (R,G,B) color in the real image -> polygon.
    """
    h, w = image_rgb.shape[:2]
    color2mask = {}
    # Convert to float to avoid overflow
    image_rgb = image_rgb.astype(np.float32)

    for y in range(h):
        for x in range(w):
            c = tuple(image_rgb[y, x])  # (R,G,B) as float
            if c not in color2mask:
                color2mask[c] = np.zeros((h, w), dtype=np.uint8)
            color2mask[c][y, x] = 1

    color2poly = {}
    for color, mask in color2mask.items():
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        polys = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < min_area:
                continue
            ep = 0.02 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, ep, True)
            coords = np.squeeze(approx).reshape(-1, 2)
            p = Polygon(coords)
            if p.is_valid and p.area >= min_area:
                polys.append(p)
        if not polys:
            continue
        merged = unary_union(polys)
        if merged.geom_type == 'MultiPolygon':
            # pick the biggest
            big = max(merged.geoms, key=lambda g: g.area)
            color2poly[color] = big
        else:
            color2poly[color] = merged

    return color2poly

###############################################################################
# Parsing the Fake Image (Threshold approach)
###############################################################################
def parse_color_with_threshold(fake_rgb, target_colors, color_dist_thresh=30):
    """
    Maps each pixel in fake_rgb to whichever 'target_colors' color is closest
    (if below 'color_dist_thresh'). Then builds a mask per color.
    """
    h, w = fake_rgb.shape[:2]
    fake_rgb = fake_rgb.astype(np.float32)

    color2mask = {}
    # Convert each target color to float
    target_colors_float = [tuple(float(x) for x in c) for c in target_colors]
    for cf in target_colors_float:
        color2mask[cf] = np.zeros((h,w), dtype=np.uint8)

    for y in range(h):
        for x in range(w):
            fr, fg, fb = fake_rgb[y, x]
            best_c = None
            best_dist = 999999.0
            for cf in target_colors_float:
                dr = fr - cf[0]
                dg = fg - cf[1]
                db = fb - cf[2]
                dist = np.sqrt(dr*dr + dg*dg + db*db)
                if dist < best_dist:
                    best_dist = dist
                    best_c = cf
            if best_dist < color_dist_thresh:
                color2mask[best_c][y, x] = 1

    return color2mask

def build_polygons_from_masks(color2mask):
    """
    Convert each mask to polygon via findContours -> merged polygon.
    """
    color2poly = {}
    for c, mask in color2mask.items():
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        polys = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area < MIN_AREA:
                continue
            ep = 0.02 * cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, ep, True)
            coords = np.squeeze(approx).reshape(-1,2)
            p = Polygon(coords)
            if p.is_valid and p.area>=MIN_AREA:
                polys.append(p)
        if not polys:
            continue
        merged = unary_union(polys)
        if merged.geom_type=='MultiPolygon':
            big = max(merged.geoms, key=lambda g: g.area)
            color2poly[c] = big
        else:
            color2poly[c] = merged
    return color2poly

###############################################################################
# 2. Rectify Both Real & Fake Polygons to 4-edge bounding rectangles
###############################################################################
def polygon_to_minAreaRect_poly(poly):
    """
    Use cv2.minAreaRect on the polygon's contour
    => returns a 4-corner rotated bounding box as a shapely Polygon
    """
    if not poly.is_valid or poly.is_empty:
        return None
    coords = np.array(poly.exterior.coords).astype(np.float32)
    cnt = coords.reshape((-1,1,2))  # contour shape
    rect = cv2.minAreaRect(cnt)   # ( (cx,cy), (w,h), angle )
    box  = cv2.boxPoints(rect)    # shape (4,2)
    return Polygon(box)

###############################################################################
# 3. Compare Rectangles (IoU, corner offsets, etc.)
###############################################################################
def iou_of_polygons(polyA, polyB):
    if polyA is None or polyB is None:
        return 0
    if not polyA.is_valid or not polyB.is_valid:
        return 0
    inter = polyA.intersection(polyB)
    union = polyA.union(polyB)
    if union.area<1e-9:
        return 0
    return inter.area / union.area

def corner_offset(rectA, rectB):
    """
    If both are valid 4-corner polygons, we can measure corner offset 
    after sorting corners by e.g. lexicographic order.
    """
    if rectA is None or rectB is None:
        return None
    if rectA.is_empty or rectB.is_empty:
        return None

    ca = list(rectA.exterior.coords)[:-1]  # 4 corners
    cb = list(rectB.exterior.coords)[:-1]
    if len(ca)!=4 or len(cb)!=4:
        return None
    # sort corners by (x,y)
    ca_sorted = sorted(ca)
    cb_sorted = sorted(cb)
    dist_sum = 0
    for (x1,y1),(x2,y2) in zip(ca_sorted, cb_sorted):
        dx = x2 - x1
        dy = y2 - y1
        dist_sum += np.hypot(dx, dy)
    return dist_sum / 4  # average corner distance

###############################################################################
# Full Pipeline for a single pair
###############################################################################
def process_fake_real_pair(
    fake_path, real_path, output_dir="results_rectify", color_dist_thresh=30
):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Load images
    fake_bgr = cv2.imread(fake_path)
    real_bgr = cv2.imread(real_path)
    if fake_bgr is None or real_bgr is None:
        print(f"Error loading {fake_path} or {real_path}")
        return pd.DataFrame()

    fake_rgb = cv2.cvtColor(fake_bgr, cv2.COLOR_BGR2RGB)
    real_rgb = cv2.cvtColor(real_bgr, cv2.COLOR_BGR2RGB)

    # 1) Parse real
    real_color2poly = parse_color_coded_image(real_rgb)  
    # 2) Build real_color2mask if needed
    real_color2mask = {}
    H, W = real_rgb.shape[:2]
    for c, poly in real_color2poly.items():
        mask = np.zeros((H,W), dtype=np.uint8)
        coords = np.array(poly.exterior.coords).round().astype(int)
        cv2.fillPoly(mask, [coords], 1)
        real_color2mask[c] = mask

    # 3) Parse fake with threshold
    real_colors = list(real_color2poly.keys())  # each is (R,G,B)
    fake_color2mask = parse_color_with_threshold(fake_rgb, real_colors, color_dist_thresh)
    fake_color2poly = build_polygons_from_masks(fake_color2mask)

    # 4) Rectify each polygon to minAreaRect
    # for both real and fake
    real_rects = {}
    for c, poly in real_color2poly.items():
        real_rects[c] = polygon_to_minAreaRect_poly(poly)

    fake_rects = {}
    for c, poly in fake_color2poly.items():
        fake_rects[c] = polygon_to_minAreaRect_poly(poly)

    # 5) Entire-image metrics (MSE, SSIM), ignoring shape constraints
    mse_val = psnr_val = ssim_val = 0
    if fake_rgb.shape == real_rgb.shape:
        fake_f = fake_rgb.astype(np.float32)/255
        real_f = real_rgb.astype(np.float32)/255
        mse_val = mean_squared_error(real_f, fake_f)
        psnr_val = peak_signal_noise_ratio(real_f, fake_f, data_range=1.0)
        try:
            ssim_val = ssim(real_f, fake_f, data_range=1.0, multichannel=True, win_size=WIN_SIZE_FOR_SSIM)
        except ValueError as e:
            print(f"SSIM error: {e}")
            ssim_val = 0

    # 6) Build final rows
    rows = []
    base_name = os.path.splitext(os.path.basename(fake_path))[0]

    # Compare for each color in real
    for c, poly_r in real_color2poly.items():
        rect_r = real_rects.get(c, None)
        # find matching color in fake
        poly_f = fake_color2poly.get(c, None)
        rect_f = fake_rects.get(c, None)

        iou_val = iou_of_polygons(rect_r, rect_f)
        corner_dist = corner_offset(rect_r, rect_f)

        # pixel count ratio if wanted
        real_cnt = np.count_nonzero(real_color2mask[c])
        fake_cnt = 0
        if c in fake_color2mask:
            fake_cnt = np.count_nonzero(fake_color2mask[c])
        ratio = fake_cnt/real_cnt if real_cnt>0 else 0

        row = {
            'file_id': base_name,
            'color': c,
            'real_area': poly_r.area if poly_r else 0,
            'fake_area': poly_f.area if poly_f else 0,
            'rect_iou': iou_val,
            'corner_offset': corner_dist,
            'pixel_ratio': ratio,
            'mse': mse_val,
            'psnr': psnr_val,
            'ssim': ssim_val,
            'fake_path': fake_path,
            'real_path': real_path
        }
        rows.append(row)

    # Also color in fake missing from real
    missing_in_real = set(fake_color2poly.keys()) - set(real_color2poly.keys())
    for c in missing_in_real:
        rect_f = fake_rects[c]
        iou_val = iou_of_polygons(None, rect_f)
        corner_dist = corner_offset(None, rect_f)
        row = {
            'file_id': base_name,
            'color': c,
            'real_area': 0,
            'fake_area': fake_color2poly[c].area if fake_color2poly[c] else 0,
            'rect_iou': iou_val,
            'corner_offset': corner_dist,
            'pixel_ratio': 0,
            'mse': mse_val,
            'psnr': psnr_val,
            'ssim': ssim_val,
            'fake_path': fake_path,
            'real_path': real_path
        }
        rows.append(row)

    # 7) Visualisation: show rects side-by-side
    fig, axes = plt.subplots(1,2, figsize=(10,6))

    # Left: fake
    axes[0].imshow(fake_rgb.astype(np.uint8))
    axes[0].set_title("Fake + rects")
    for c, rp in fake_rects.items():
        if rp and rp.is_valid:
            coords = np.array(rp.exterior.coords)
            axes[0].plot(coords[:,0], coords[:,1], 'r', linewidth=1)

    # Right: real
    axes[1].imshow(real_rgb.astype(np.uint8))
    axes[1].set_title("Real + rects")
    for c, rp in real_rects.items():
        if rp and rp.is_valid:
            coords = np.array(rp.exterior.coords)
            axes[1].plot(coords[:,0], coords[:,1], 'g', linewidth=1)

    out_fig = os.path.join(output_dir, f"{base_name}_rects_side_by_side.png")
    plt.tight_layout()
    fig.savefig(out_fig, dpi=150, bbox_inches='tight')
    plt.close(fig)

    return pd.DataFrame(rows)

def main():
    """
    Demo: parse real vs. fake, rectify both to minAreaRect, compare.
    """
    fake_path = "../data/ny-brooklyn/ma-boston-p2p-500-150-v100/test_latest_500e-Brooklyn/images/combined_200035_fake_B.png"
    real_path = "../data/ny-brooklyn/ma-boston-p2p-500-150-v100/test_latest_500e-Brooklyn/images/combined_200035_real_B.png"
    out_dir = "results_rectify"

    df = process_fake_real_pair(fake_path, real_path, out_dir, color_dist_thresh=30)
    if not df.empty:
        csv_path = os.path.join(out_dir, "detailed_rectified.csv")
        df.to_csv(csv_path, index=False)
        print(f"Saved rectified comparison to {csv_path}")
    else:
        print("No data produced")

if __name__=="__main__":
    main()


Saved rectified comparison to results_rectify/detailed_rectified.csv
